In [1]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeVectorStore
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers
import pypdf

c:\Users\kkmax\anaconda3\envs\mchatbotEnv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
#EXTRACT DATA FROM THE PDF
def load_pdf(data):
    loader = DirectoryLoader(data, glob="*.pdf", loader_cls=PyPDFLoader)
    documents = loader.load()
    return documents

In [3]:
extracted_data = load_pdf("data/")

In [4]:
#CREATING TEXT CHUNKS
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks

In [5]:
text_chunks = text_split(extracted_data)

In [6]:
print(len(text_chunks))

7020


In [7]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [8]:
embeddings = download_hugging_face_embeddings()

In [9]:
query_result = embeddings.embed_query("hello world")
print("length" , len(query_result))

length 384


In [14]:
PINECONE_API_KEY = 'fd8264ea-1261-4159-94c1-416aa5c6778b'
PINECONE_API_ENV = 'gcp-starter'

In [17]:
import os
os.environ['PINECONE_API_KEY'] = PINECONE_API_KEY

In [18]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

index_name = "medical-chatbot"
if index_name not in pc.list_indexes().names():
  # Do something, such as create the index
  pc.create_index(
    name=index_name,
    dimension=384,
    metric='cosine',
    spec=ServerlessSpec(
      cloud="gcp-starter",
      region="us-central1"
    )
  )

In [19]:
docsearch = PineconeVectorStore.from_texts([t.page_content for t in text_chunks], embeddings, index_name="medical-chatbot")

In [22]:
query = "What are Allergies"

docs=docsearch.similarity_search(query, k=3)

print("Result", docs[0].page_content)

Result GALE ENCYCLOPEDIA OF MEDICINE 2 117Allergies
Allergic rhinitis is commonly triggered by
exposure to household dust, animal fur,or pollen. The foreign substance thattriggers an allergic reaction is calledan allergen.
The presence of an allergen causes the
body's lymphocytes to begin producingIgE antibodies. The lymphocytes of an allergy sufferer produce an unusuallylarge amount of IgE.
IgE molecules attach to mast
cells, which contain histamine.HistaminePollen grains
Lymphocyte
FIRST EXPOSURE


In [23]:
prompt_template="""
Use the following pieces of information to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer.

Context: {context}
Question: {question}

Only return the helpful answer below and nothing else.
Helpful answer:
"""

In [24]:
PROMPT=PromptTemplate(template=prompt_template, input_variables=["context", "question"])
chain_type_kwargs={"prompt": PROMPT}

In [25]:
llm=CTransformers(model="model/llama-2-7b-chat.ggmlv3.q4_0.bin",
                  model_type="llama",
                  config={'max_new_tokens':512,
                          'temperature':0.8})

In [26]:

qa=RetrievalQA.from_chain_type(
    llm=llm, 
    chain_type="stuff", 
    retriever=docsearch.as_retriever(search_kwargs={'k': 2}),
    return_source_documents=True, 
    chain_type_kwargs=chain_type_kwargs)

In [27]:
while True:
    user_input=input(f"Input Prompt:")
    result=qa({"query": user_input})
    print("Response : ", result["result"])

c:\Users\kkmax\anaconda3\envs\mchatbotEnv\lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


Response :  The prognosis varies with the cause of hair loss. If the hair loss is due to a medical condition, such as alopecia areata or thyroid disorders, treatment may involve medication or other therapies to manage the underlying condition. In some cases, hair growth may return spontaneously once the condition is treated. However, if the hair loss is due to aging or genetics, there is no known cure and hair growth may not return.



